**Łukasz Folwarczyk**

*"Badanie i implementacja algorytmów detekcji drogi przy wykorzystaniu cyfrowego przetwarzania obrazów."*

---

# Segmentacja park tuning


Notatnik implementujący architekturę głębokiej sieci neuronowej, wykorzystanej do problemu segmentacji obrazów przedstawiających widoki dróg parkowych. Działanie modułu opisane zostało w paragrafie 5.1.1 - przypadek III.

In [ ]:
%matplotlib inline
import scipy
import os, sys
import cv2
import numpy as np
import skimage.io as io
import PIL.Image
import tensorflow as tf
import time
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras import optimizers
from keras import backend as K
from keras.callbacks import Callback
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import CSVLogger
from keras.callbacks import TerminateOnNaN
sys.path.append('../../../Tools/cropping2d')
from croppingLike2D import CroppingLike2D
import math
from random import shuffle
from math import ceil
from random import randint
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
import scipy.misc
import gc
import datetime
import h5py
from IPython.display import clear_output

timefunc = time.time
assert K.backend() == 'tensorflow'

In [ ]:
def batch_generator(hdf5_filename, 
                    dataset_img, 
                    dataset_label, 
                    one_hot, 
                    batch_size, 
                    shuffle_batch = True):
    """ 
    Opis:
    ---
    Funkcja pełniąca rolę generatora próbek (mini-paczek) uczących oraz walidacyjnych.
    Wykorzystywana w procesie uczenia głębokiej sieci neuronowej.
    
    Argumenty:
    ---
        hdf5_filename (string):   nazwa pliku zawierającego zbiór danych
        
        dataset_img (string):     nazwa zbioru pliku hdf5, 
                                  w którym zapisane sa obrazy
        
        dataset_label (string):   nazwa zbioru pliku hdf5, 
                                  w którym zapisane sa etykiety
        
        one_hot (boolean):        kodowanie gorąco-jedynkowe
                                                                  
        batch_size (int):         ilość generowanych próbek (wielkość mini-paczki)
        
        shuffle_batch (int):      losowe próbki ze zbioru
    
    Return:
    ---
        generator (truple):       mini-paczka zawierająca obrazy oraz ich etykiety
    """
    # otwarcie pliku hdf5
    with h5py.File(hdf5_filename) as hdf_file_read:                                 
        # wyznaczenie wielkosci zbioru zapisanego w pliku
        dataset_len = hdf_file_read[dataset_img].shape[0]                          
        # liczba mini-paczek możliwych do wygenerowania z danego zbioru
        number_of_batches = int(ceil(float(dataset_len) / batch_size)) #ilosc batchy = foty / pojemnosc_batcha
        batches_list = list(range(number_of_batches))
        # losowość min-paczki
        if shuffle_batch:
            shuffle(batches_list)
        i = 0
        while True:
            # wyznaczenie indeksu początkowego i końcowego obrazów oraz etykiet,
            # które zostaną odczytane z pliku i przekazane w postaci mini-paczki
            first_image_in_batch_index = batches_list[i] * batch_size
            last_image_in_batch_index = min([(batches_list[i] + 1) * batch_size, dataset_len])
            # odczytanie z pliku obrazów i etykiet o wyznaczonych indeksach
            reconstructed_img = (hdf_file_read[dataset_img][first_image_in_batch_index:last_image_in_batch_index, ...])
            reconstructed_label = (hdf_file_read[dataset_label][first_image_in_batch_index:last_image_in_batch_index, ...])
            # przejście przez cały zbiór obrazów/etykiet zapisany w pliku (pokazanie wszystkich próbek)
            # jeśli w min-paczkach pokazano wszystkie próbki to rozpoczni od nowa, 
            # jesli wybrano shuffle_batch to w każdym nowym cyklu próbki są generowane w losowej kolejności
            if i < (number_of_batches - 1):
                i += 1
            else:
                i = 0
                if shuffle_batch:
                    shuffle(batches_list)
            # generowanie mini-paczek
            yield reconstructed_img.astype(np.float32), reconstructed_label.astype(np.float32)

In [ ]:
def batch_generator_test(hdf5_filename, 
                         dataset_img, 
                         batch_size, 
                         shuffle_batch = True):
    """ 
    Opis:
    ---
    Funkcja pełniąca rolę generatora próbek (mini-paczek) testowych (nie zawierają etykiet).
    Wykorzystywana w procesie testowania głębokiej sieci neuronowej.
    
    Argumenty:
    ---
        hdf5_filename (string):   nazwa pliku zawierającego zbiór danych
        
        dataset_img (string):     nazwa zbioru pliku hdf5, 
                                  w którym zapisane sa obrazy
                                                                  
        batch_size (int):         ilość generowanych próbek (wielkość mini-paczki)
        
        shuffle_batch (int):      losowe próbki ze zbioru
    
    Return:
    ---
        generator (truple):       mini-paczka zawierająca obrazy
    """
    # otwarcie pliku hdf5
    with h5py.File(hdf5_filename) as hdf_file_read:
        # wyznaczenie wielkosci zbioru zapisanego w pliku
        dataset_len = hdf_file_read[dataset_img].shape[0]
        # liczba mini-paczek możliwych do wygenerowania z danego zbioru
        number_of_batches = int(ceil(float(dataset_len) / batch_size))
        batches_list = list(range(number_of_batches))
        # losowość min-paczki
        if shuffle_batch:
            shuffle(batches_list)
        i = 0
        while True:
            # wyznaczenie indeksu początkowego i końcowego obrazów,
            # które zostaną odczytane z pliku i przekazane w postaci mini-paczki
            first_image_in_batch_index = batches_list[i] * batch_size  # index of the first image in this batch
            last_image_in_batch_index = min([(batches_list[i] + 1) * batch_size, dataset_len])
            # odczytanie z pliku obrazów o wyznaczonych indeksach
            reconstructed_img = (hdf_file_read[dataset_img][first_image_in_batch_index:last_image_in_batch_index, ...])
            # przejście przez cały zbiór obrazów/etykiet zapisany w pliku (pokazanie wszystkich próbek)
            # jeśli w min-paczkach pokazano wszystkie próbki to rozpoczni od nowa, 
            # jesli wybrano shuffle_batch to w każdym nowym cyklu próbki są generowane w losowej kolejności
            if i < (number_of_batches - 1):
                i += 1
            else:
                i = 0
                if shuffle_batch:
                    shuffle(batches_list)
            # generowanie mini-paczek
            yield reconstructed_img.astype(np.float32)

In [ ]:
def normalize_img(image, image_type):
    """ 
    Opis:
    ---
    Funkcja normalizująca obraz.
    
    Argumenty:
    ---
        image (numpy.ndarray):    obraz wejściowy
        
        image_type (numpy.dtype): typ obrazu wejściowego
    
    Return:
    ---
        image (numpy.ndarray):    obraz znormalizowany
    """
    mean = image.mean()
    image = image - mean
    image /= (image.std() + 1e-5)
    image *= 0.1
    image += 0.5
    image = np.clip(image, 0, 1)
    image *= 255
    image = image.astype(image_type)
    return image

In [ ]:
def print_layer_trainable(model):
    """ 
    Opis:
    ---
    Funkcja wyświetla nazwy wszystkich warstw obecnych w modelu i
    informacje o tym, czy podlegają one procesowi uczenia.
    
    Argumenty:
    ---
        model (keras.engine.training.Model):   nazwa pliku zawierającego zbiór danych
    
    Return:
    ---
        None
    """
    for layer in model.layers:
        print("{0}: \t{1}".format(layer.trainable, layer.name))

In [ ]:
def display_img_batch(hdf5_filename, 
                      dataset_img, 
                      dataset_label, 
                      one_hot, 
                      batch_size, 
                      class_index):  
    """ 
    Opis:
    ---
    Funkcja wyświetla przykładowe obrazy oraz ich etykiety, 
    zapisane w pliku z rozszerzeniem hdf5.
    
    Argumenty:
    ---
        hdf5_filename (string):   nazwa pliku wynikowego
        
        dataset_img (string):     nazwa zbioru pliku hdf5, 
                                  w którym zapisane sa obrazy
        
        dataset_label (string):   nazwa zbioru pliku hdf5, 
                                  w którym zapisane sa etykiety
        
        one_hot (boolean):        kodowanie gorąco-jedynkowe
                                                                  
        batch_size (int):         ilosć wyświetlonych próbek
        
        class_index (int):        numer klasy wyróżnionej na zdjęciu
    
    Return:
    ---
        None
    """
    with h5py.File(hdf5_filename) as hdf_file_read:                  # otwarcie pliku hdf5
        height = hdf_file_read["height"][0, ...]                     # odczytanie wysokosci obrazów/etykiet
        width = hdf_file_read["width"][0, ...]                       # odczytanie szerokoci obrazów/etykiet
        cls =  hdf_file_read["classes"][0, ...]                      # odczytanie ilości klas w etykietach
        dataset_len = hdf_file_read[dataset_img].shape[0]            # odczytanie ilości par obraz/etykieta
        
        # wyświetlenie par obraz/etykieta (ilość zależna od parametru batch_size)
        for i in range(batch_size):
            # zostaje wyświetlona losowa para
            rand = randint(0, dataset_len)
            
            # odtworzenie obrazu / etykiety
            reconstructed_img = (hdf_file_read[dataset_img][rand:min(rand + 1,dataset_len), ...])
            reconstructed_img = reconstructed_img[0,:,:,:]
            img_shape = reconstructed_img.shape
            reconstructed_label = (hdf_file_read[dataset_label][rand:min(rand + 1,dataset_len), ...])
            
            # przetworzenie etykiety w zalezności od kodowania
            if one_hot:
                reconstructed_label = reconstructed_label[0,:,:]
                label_shape = reconstructed_label.shape
                reconstructed_label = reconstructed_label.reshape(height,width,cls)
            else:
                reconstructed_label = reconstructed_label[0,:,:,:]
                label_shape = reconstructed_label.shape
             
            # wyświetlenie pary obraz/etykieta
            fig = plt.figure()
            fig.add_subplot(1, 2, 1)
            plt.imshow(reconstructed_img)
            fig.add_subplot(1, 2, 2)
            plt.imshow(reconstructed_label[:,:,class_index])
    print("---------------------------".format(dataset_len))
    print("Ilość obrazów w zbiorze: {}".format(dataset_len))
    print("Kształt obrazów: {}".format(img_shape))
    print("Kształt etykiet: {}".format(label_shape))

In [ ]:
def number_of_examples_in_dataset(hdf5_filename,
                                  dataset_img,
                                  dataset_label):
    """ 
    Opis:
    ---
    Funkcja wyznacza ilośc próbek w zbiorze danych (zapisanym w pliku hdf5).
    
    Argumenty:
    ---
        hdf5_filename (string):   nazwa pliku zawierającego zbiór danych
        
        dataset_img (string):     nazwa zbioru pliku hdf5, 
                                  w którym zapisane sa obrazy
        
        dataset_label (string):   nazwa zbioru pliku hdf5, 
                                  w którym zapisane sa etykiety
    
    Return:
    ---
        None
    """
    with h5py.File(hdf5_filename) as hdf_file_read:                  # otwarcie pliku hdf5
        dataset_len_img = hdf_file_read[dataset_img].shape[0]        # odczytanie ilości obrazów w zbiorze
        dataset_len_labels = hdf_file_read[dataset_label].shape[0]   # odczytanie ilości etykiet w zbiorze
        
        # sprawdzenie czy ilość obrazów i etykiet jest zgodna, jeśli nie - BŁĄD
        assert dataset_len_img == dataset_len_labels, "Ilośc obrazów jest niezgodna z ilością etykiet!"
        return dataset_len_img
        

In [ ]:
def generate_examples(test_model, 
                      number_of_exaples_to_generate, 
                      generator, 
                      exaples_dir):
    """ 
    Opis:
    ---
    Funkcja generuje pliki png przedstawiajace obraz oryginalny, 
    przewidywania sieci oraz etykietę (na jednym obrazie).
    
    Argumenty:
    ---
        test_model (keras.engine.training.Model):   model sieci neuronowej z załadowanymi wagami
        
        number_of_exaples_to_generate (string):     ilość generowanych przykładw (plików png) 
                                                    w którym zapisane sa obrazy
        
        generator (generator):                      generator próbek (obraz + etykieta)
                                                                  
        exaples_dir (string):                       nazwa katalogu wyjściowego
    
    Return:
    ---
        None
    """
    plt.ioff()
    # dla każdej próbki uczącej
    for number in range(number_of_exaples_to_generate):
        img, label = generator.__next__()                             # pobranie obrazu i etykiety
        y_pred = test_model.predict(x = img)                          # dokonanie przewidywań dla obrazu
        y_pred = y_pred[0].reshape(image_height, image_width, 2)      # przekształcenie z kodowania gorąco-jedynkowego 
                                                                      # (przewidywania)  
        label = label.reshape(image_height, image_width, 2)           # przekształcenie z kodowania gorąco-jedynkowego
                                                                      # (etykieta)
        img_org = img.copy()

        # jeśli prawdopodobieństwo przynależności piksela do klasy droga > 50% 
        # to oznacz piksel na zielono
        for i in range(image_height):
            for j in range(image_width):
                if y_pred[i,j,0] > 0.2 and y_pred[i,j,0] < 0.5:
                    img[0,i,j,0] += 0
                elif y_pred[i,j,0] > 0.5 and y_pred[i,j,0] < 0.9:
                    img[0,i,j,1] = 250
                elif y_pred[i,j,0] > 0.90:
                    img[0,i,j,1] = 250
        
        # stworzenie obrazu przedstawiajacego obraz oryginalny, przewidywania, etykietę
        fig = plt.figure(figsize=(14, 12), dpi = image_height)
        fig.add_subplot(1, 3, 1)
        plt.imshow(img_org[0,:,:,:].astype(np.uint8))
        fig.add_subplot(1, 3, 2)
        plt.imshow(img[0,:,:,:].astype(np.uint8))
        fig.add_subplot(1, 3, 3)
        plt.imshow(label[:,:,0].astype(np.uint8))

        # zapis obrazu do pliku png
        figure_name = exaples_dir + 'example_{}'.format(number + 1) 
        plt.savefig(figure_name)
        plt.close(fig)
        gc.collect()
        clear_output()
        print('Zapisanych: {}/{}'.format(number + 1, number_of_exaples_to_generate))

In [ ]:
def calculate_predict_time(test_model,
                           number_of_exaples_to_generate, 
                           generator):
    """ 
    Opis:
    ---
    Funkcja dokonuje przewidywań sieci dla number_of_exaples_to_generate próbek
    i wyznacza czas potrzebny do dokonania przewidywań dla pojedynczej prbki.
    (jako średnia dla number_of_exaples_to_generate próbek)
    
    Argumenty:
    ---
        test_model (keras.engine.training.Model):   model sieci neuronowej z załadowanymi wagami
        
        number_of_exaples_to_generate (string):     ilość generowanych przykładw (plików png) 
                                                    w którym zapisane sa obrazy
        
        generator (generator):                      generator próbek (obraz + etykieta)
                                                                
    Return:
    ---
        None
    """
    time_all = 0
    # dla każdej prbki
    for number in range(number_of_exaples_to_generate):
        img = generator.__next__()                                 # pobranie zdjcia
        start_time = time.time()                                   # rozpoczęcie odmierzania czasu
        y_pred = test_model.predict(x = img)                       # dokonanie przewidywań
        stop_time = time.time()                                    # zakończenie odmierzania czasu
        pred_time = stop_time - start_time                         # wyznaczenie czasu przewidywań dla pojedynczej próbki
        time_all += pred_time                                      # czas sumaryczny dla wszytskich próbek
    time_for_single_example = time_all / number_of_exaples_to_generate  # średnia przewidywań
    
    print("Czas przewidywań dla pojedynczej próbki: {:.4f} s (średnia dla {} próbek)."\
          .format(time_for_single_example, number_of_exaples_to_generate))


In [ ]:
def VGG16(model_input, weights = 'imagenet', weight_decay = 0.):
    """ 
    Opis:
    ---
    Funkcja implementująca architekturę VGG16, bez ostatnich warstw w pełni połączonych.
    
    Argumenty:
    ---
        model_input (tensorflow.python.framework.ops.Tensor):   warstwa wejciowa (tensor)
        
        weights (string):                                       załadowanie wag imagenet
                                
        weight_decay (string):                                  wartosć współczynnik regularizacji
    
    Return:
    ---
        truple:                                                 krotka, ktrej elementy są typu
                                                                tensorflow.python.framework.ops.Tensor
                                                                (3,4,5 warstwa buforowa modelu)
    """
    # pierwsza warstwa konwolucyjna
    x = Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block1_conv1',
               data_format = 'channels_last', kernel_regularizer = l2(weight_decay))(model_input)   
    
    x = Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block1_conv2',
               data_format = 'channels_last', kernel_regularizer = l2(weight_decay))(x)   
    
    # pierwsza warstwa buforowa
    x = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same', 
               name = 'block1_pool', data_format = 'channels_last')(x)

    
    # druga warstwa konwolucyjna
    x = Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block2_conv1',
               data_format = 'channels_last', kernel_regularizer = l2(weight_decay))(x)   
    
    x = Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block2_conv2',
               data_format = 'channels_last', kernel_regularizer = l2(weight_decay))(x)  
    
    # druga warstwa buforowa
    x = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same', 
               name = 'block2_pool', data_format = 'channels_last')(x)

    
    # trzecia warstwa konwolucyjna
    x = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv1',
               data_format = 'channels_last', kernel_regularizer = l2(weight_decay))(x)  
    
    x = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv2',
               data_format = 'channels_last', kernel_regularizer = l2(weight_decay))(x)  
    
    x = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block3_conv3',
               data_format = 'channels_last', kernel_regularizer = l2(weight_decay))(x)  
    
    # trzecia warstwa buforowa
    x = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same', 
               name = 'block3_pool', data_format = 'channels_last')(x)
    
    # pierwszy element zwracanej krotki = trzecia warstwa buforowa
    pool3 = x
    
    
    # czwarta warstwa konwolucyjna
    x = Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv1',
               data_format = 'channels_last', kernel_regularizer = l2(weight_decay))(x)  
    
    x = Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv2',
               data_format = 'channels_last', kernel_regularizer = l2(weight_decay))(x) 
    
    x = Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block4_conv3',
               data_format = 'channels_last', kernel_regularizer = l2(weight_decay))(x) 
    
    # czwarta warstwa buforowa
    x = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same', 
               name = 'block4_pool', data_format = 'channels_last')(x)

    # drugi element zwracanej krotki = czwarta warstwa buforowa
    pool4 = x
    
    
    # piąta warstwa konwolucyjna
    x = Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv1',
               data_format = 'channels_last', kernel_regularizer=l2(weight_decay))(x) 
    
    x = Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv2',
               data_format = 'channels_last', kernel_regularizer=l2(weight_decay))(x) 
    
    x = Conv2D(filters = 512, kernel_size = (3, 3), activation = 'relu', padding = 'same', name = 'block5_conv3',
               data_format = 'channels_last', kernel_regularizer=l2(weight_decay))(x) 
    
    # trzeci element zwracanej krotki = piąta warstwa buforowa
    x = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same', 
               name = 'block5_pool', data_format = 'channels_last')(x)
    
    
    # załadowanie wag ImageNet
    model_temp = Model(inputs = model_input, outputs = x)
    if weights is not None:
        model_temp.load_weights(vgg16_weights_file)
        
    layers = (pool3, pool4, x)

    return layers

In [ ]:
def FCN8(model_out, pool3, pool4, pool5, classes, dropout = 0.5, weight_decay = 0.):  
    """ 
    Opis:
    ---
    Funkcja implementująca architekturę FCN8.
    
    Argumenty:
    ---
        model_out (truple):                              rozmiat tensora reprezentującego warstwę wyjściową
        
        pool3 (tensorflow.python.framework.ops.Tensor):  trzecia warstwa buforowa modelu VGG16
                     
        pool4 (tensorflow.python.framework.ops.Tensor):  czwarta warstwa buforowa modelu VGG16
        
        pool5 (tensorflow.python.framework.ops.Tensor):  piąta warstwa buforowa modelu VGG16
                     
        classes (int):                                   liczba klas w zbiorze danych
        
        dropout (float):                                 wartość współczynnika dropout
        
        weight_decay (string):                           wartosć współczynnik regularizacji
    
    Return:
    ---
        tensorflow.python.framework.ops.Tensor:          warstwa wyjściowa modelu
                                                                
    """
    # szósta warstwa konwolucyjna
    x = Conv2D(filters = 4096, kernel_size = (7, 7), activation = 'relu', padding = 'same', 
                 name = 'block6_conv1', data_format = 'channels_last')(pool5)
    # warstwa dropout
    x = Dropout(rate = dropout)(x)
    
    # siódma warstwa konwolucyjna
    x = Conv2D(filters = 4096, kernel_size = (1, 1), activation = 'relu', padding = 'same', 
               name = 'block6_conv2', data_format = 'channels_last')(x)
    # warstwa dropout
    x = Dropout(rate = dropout)(x)

    # wagi dla przewidywań 
    weight_pool5, weight_pool4, weight_pool3 = [1, 1e-2, 1e-4]
    
    # ----------------------
    
    # nadanie wagi dla przewidywań (związanych z warstwą pool5) 
    x = Lambda(lambda xx: xx * weight_pool5, name = 'weight_pool5')(x)
    
    # warstwa konwolucyjna (związana z warstwą pool5)
    x = Conv2D(filters = classes, kernel_size = (1, 1), padding = 'valid', 
               kernel_initializer = 'he_normal', 
               name = 'score_feat1', data_format = 'channels_last', 
               activation='linear', kernel_regularizer = l2(weight_decay))(x)
    
    # warstwa dekonwolucyjna (związana z warstwą pool5)
    y = Conv2DTranspose(filters = classes, kernel_size = (4, 4), strides = (2, 2), use_bias = False, 
                padding = 'valid', name = 'upscore_feat1', data_format = 'channels_last')(x)
    
    # ----------------------
    
    # nadanie wagi dla przewidywań (związanych z warstwą pool4) 
    pool4 = Lambda(lambda xx: xx * weight_pool4, name='weight_pool4')(pool4)
    
    # warstwa konwolucyjna (związana z warstwą pool4)
    pool4 = Conv2D(filters = classes, kernel_size = (1, 1), padding = 'valid', 
                   kernel_initializer = 'he_normal', 
                   name = 'score_feat2', data_format = 'channels_last', 
                   activation='linear', kernel_regularizer = l2(weight_decay))(pool4)
    
    # dopasowanie wymiarów tensorów (związanego z warstwą dekonwolucyjną, związaną z warstwą pool5 
    # i konwolucyjną związaną z warstwą pool4)
    y = CroppingLike2D(target_shape = K.int_shape(pool4), data_format = 'channels_last', 
                       offset='centered', name = 'crop_feat2')(y)
    
    # dodanie przewidywań
    y = Add()([y, pool4])
    
    # warstwa dekonwolucyjna
    y = Conv2DTranspose(filters = classes, kernel_size = (4, 4), strides = (2, 2), use_bias = False, 
                padding = 'valid', name = 'upscore_feat2', data_format = 'channels_last')(y)
    
    # ----------------------
    
    # nadanie wagi dla przewidywań (związanych z warstwą pool3) 
    pool3 = Lambda(lambda xx: xx * weight_pool3, name='weight_pool3')(pool3)
    
    # warstwa konwolucyjna (związana z warstwą pool3)
    pool3 = Conv2D(filters = classes, kernel_size = (1, 1), padding='valid' , 
                   kernel_initializer = 'he_normal', 
                   name = 'score_feat3', data_format = 'channels_last',
                   activation='linear', kernel_regularizer = l2(weight_decay))(pool3)
   
    # dopasowanie wymiarów tensorów 
    y = CroppingLike2D(target_shape = K.int_shape(pool3), data_format = 'channels_last',
                       offset='centered', name = 'crop_feat3')(y)
    
    # dodanie przewidywań
    y = add([y, pool3])
    
    # warstwa dekonwolucyjna
    y = Conv2DTranspose(filters = classes ,kernel_size = (16,16), strides = (8,8), use_bias = False, 
                padding = 'valid', name = 'upscore_feat3', data_format = 'channels_last')(y)
    
    # dopasowanie wymiaru tensora przewidywań do wymaganych przewidywań (podanych jako parametr)
    outputs = CroppingLike2D(target_shape = model_out, data_format = 'channels_last', name = 'score')(y)  
    
    # ----------------------
    
    # kodowanie gorąco-jedynkowe
    outputs = Reshape((image_height*image_width, classes), 
                       input_shape = (image_height,image_width,classes))(outputs)

    # funkcja softmax
    scores = Activation('softmax', name='segmentation')(outputs)
        
    return scores 

---

# Przykład

In [ ]:
uczenie = "seg_park_tr"

#### Pliki wejściowe

In [ ]:
tools_dir = "../../../Tools/"

# pliki hdf5
hdf5_filename_train = tools_dir + 'hdf5/park_train_200_300.hdf5'
hdf5_filename_eval = tools_dir + 'hdf5/park_eval_200_300.hdf5'
hdf5_filename_test = tools_dir + 'hdf5/park_test_200_300.hdf5'

# plik h5 z wagami ImageNet
vgg16_weights_file = tools_dir + 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

#### Pliki wyjściowe

In [ ]:
# stworzenie katalogu, w którym zapisane zostaną pliki związane z procesem uczenia
curr_dir =  os.getcwd()
out_files_path = curr_dir + '/' + "uczenie_out/"
now = datetime.datetime.now()
if not os.path.exists(out_files_path):
    os.makedirs(out_files_path)
out_files_path += str(now.day) + '-' + str(now.month) + '-' + str(now.year)

# stworzenie katalogu, w którym zapisany zostanie plik wag nauczonego modelu
out_weights_dir = out_files_path + '/' + "weights/"
if not os.path.exists(out_weights_dir):
    os.makedirs(out_weights_dir)
output_weights_file = out_weights_dir + uczenie + "_" + str(now.hour) + '-' + str(now.minute) + ".h5" 

# stworzenie katalogu, w którym zapisany zostanie plik csv, związany z procesem cuzenia
out_csv_dir = out_files_path + '/' + "csv/"    
if not os.path.exists(out_csv_dir):
    os.makedirs(out_csv_dir)  
output_csv_file = out_csv_dir + uczenie + "_" +  str(now.hour) + '-' + str(now.minute) + ".csv"      

# stworzenie katalogu, w którym zapisany zostanie plik log, związany z procesem cuzenia
out_log_dir = out_files_path + '/' + "log/"    
if not os.path.exists(out_log_dir):
    os.makedirs(out_log_dir)  
output_log_file = out_log_dir + uczenie + "_" +  str(now.hour) + '-' + str(now.minute) + ".log" 

with open(out_files_path + "/README.txt", 'w', encoding='utf-8') as file:
    file.write("""Katalogi generowane automatycznie. Zostaną uzupełnione plikami tylko jeśli
komórki związane z procesem uczenia zostaną odkomentowane.""")

learning_start = now

#### Podstawowe parametry procesu uczenia

In [ ]:
image_height = 200
image_width = 300
batch_size = 2
img_depth = 3
label_depth = 1
epochs = 1                                                    # domyślnie tylko 1 epoka! 
img_shape = (image_height, image_width, img_depth)
new_num_classes = 2
out_size = (None, image_height, image_width, new_num_classes)
optimizer = "sgd"
LR = 0.001 
DECAY = 0.0005 
MOMENTUM = 0.9
LOSS = 'categorical_crossentropy'

#### Przykładowe próbki  zbioru uczącego

In [ ]:
display_img_batch(hdf5_filename_train, 
                  "imgs", 
                  "labels", 
                  batch_size = 2,
                  one_hot = True,
                  class_index = 0)
                
train_len = number_of_examples_in_dataset(hdf5_filename_train, "imgs", "labels")                  

#### Przykładowe próbki  zbioru walidacyjnego

In [ ]:
display_img_batch(hdf5_filename_eval, 
                  "imgs", 
                  "labels", 
                  batch_size = 2, 
                  one_hot = True,
                  class_index = 0)

eval_len = number_of_examples_in_dataset(hdf5_filename_eval, "imgs", "labels")   

#### Przykładowe próbki  zbioru testowego

In [ ]:
display_img_batch(hdf5_filename_test, 
                  "imgs", 
                  "labels", 
                  batch_size = 2, 
                  one_hot = True,
                  class_index = 0)

test_len = number_of_examples_in_dataset(hdf5_filename_test, "imgs", "labels")  

#### Sprawdzenie poprawności działania funkcji batch_generator

In [ ]:
G = batch_generator(hdf5_filename_train, 
                    "imgs", 
                    "labels", 
                    batch_size = batch_size, 
                    one_hot = True)

In [ ]:
(img, label) = G.__next__()
label = label[0,:,:].reshape((image_height, image_width, new_num_classes))
fig=plt.figure()
fig.add_subplot(1, 2, 1)
plt.imshow(img[0,:,:,:].astype(np.uint8))
fig.add_subplot(1, 2, 2)
plt.imshow(label[:,:,0])

In [ ]:
gc.collect()

---

# Należy odkomentować uczenie badź testowanie

# Uczenie modelu

#### Stworzenie modelu do uczenia

#### Kompilacja modelu

#### Funkcje wywoływane w trakcie procesu uczenia

#### Stworzenie generatorów próbek

#### Proces uczenia

#### Zapis histori uczenia do pliku, po zakończeniu uczenia

---

# Testowanie modelu
Przed uruchomieniem konieczne jest odkomentowanie poniższych komórek, zakomentowanie tych związanych z procesem uczenia i ponowne uruchomienie notatnika (Kernel > Restart and run all (zwykle x2))

#### Stworzenie modelu testowego i wczytanie wag wyznaczonych w procesie uczenia

In [ ]:
# ścieżka do pliku zawierającego nauczone wagi
model_weights_file = "seg_park_ft_weights.h5"

In [ ]:
x_test_inp = Input(shape = img_shape, dtype = 'float32')
pool3, pool4, pool5 = VGG16(x_test_inp, weights = "None")
test_model_output = FCN8(out_size, pool3, pool4, pool5, new_num_classes)
test_model = Model(inputs = x_test_inp, outputs = test_model_output)
test_model.load_weights(model_weights_file)

In [ ]:
test_model.summary()

In [ ]:
sgd = optimizers.SGD(lr = 0.001, 
                     decay = 0.0005, 
                     momentum = 0.9)
                     
test_model.compile(optimizer = sgd,
                   loss = 'categorical_crossentropy',
                   metrics = ['accuracy'])

#### Generator testowy

In [ ]:
G3 = batch_generator(hdf5_filename_test, 
                     "imgs", 
                     "labels", 
                     batch_size = 1, 
                     one_hot = True,
                     shuffle_batch = False)

#### Sprawdzenie modelu

In [ ]:
result = test_model.evaluate_generator(G3, steps = 110)

In [ ]:
for name, value in zip(test_model.metrics_names, result):
    print(name, value)
print("{0}: {1:.2%}".format(test_model.metrics_names[1], result[1]))

#### Wizualizacja wynikow

In [ ]:
# Dokonanie przewidywań dla obrazów i zapis wyniku w postaci plików png 
# przedstawiających obraz oryginalny, przewidywania sieci oraz etykietę. 

# ilość stworzonych plików png
number_of_exaples_to_generate = 5

# katalog, w kótrym powstaną pliki wynikowe
exaples_dir = os.getcwd() + "/examples/"
if not os.path.exists(exaples_dir):
    os.makedirs(exaples_dir)

# generator testowy 
G4 = batch_generator(hdf5_filename_test, 
                     "imgs", 
                     "labels", 
                     batch_size = 1, 
                     one_hot = True,
                     shuffle_batch = False)    

generate_examples(test_model, number_of_exaples_to_generate, G4, exaples_dir)

In [ ]:
# Wyznaczenie czasu potrzebnego na dokonanie przewidywań dla pojedynczej próbki 
# (jako średnia dla number_of_exaples_to_generate)

#ilość próbek, dla których wyznaczona zostanie średnia
number_of_exaples_to_generate = 100

# generator testowy 
G5 = batch_generator_test(hdf5_filename_test, 
                  "imgs", 
                  batch_size = 1)

calculate_predict_time(test_model, number_of_exaples_to_generate, G5)